In [14]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout
import numpy
import pandas

In [15]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer 

#### Prepare data

In [16]:
trainmatchups = pandas.read_csv('kerasfiles/train.matchups.ODDeltaStats.csv')
print(trainmatchups.shape)
traindata = trainmatchups.values

(25844, 67)


In [17]:
traindata = trainmatchups.values

trainX = traindata[:,1:]
trainY = traindata[:,0]

print(trainX.shape)
print(trainY.shape)

input_shape = trainX.shape[1]

(25844, 66)
(25844,)


#### Create Model

In [18]:
def create_model(dropout=0.2,kernel_initializer='normal'):
    model = Sequential()
    model.add(Dense(256, kernel_initializer=kernel_initializer, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(dropout))
    model.add(Dense(128, kernel_initializer=kernel_initializer, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(64, kernel_initializer=kernel_initializer, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(16, kernel_initializer=kernel_initializer, activation='relu'))
    model.add(Dense(1, kernel_initializer=kernel_initializer, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
model = create_model(0.1)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               17152     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                1040      
__________

#### Prepare data

In [20]:
#scaler = StandardScaler(with_mean=False, with_std=False) # no scaling
#trainX_scaled = trainX
scaler = StandardScaler()
trainX_scaled = trainX
#scaler = StandardScaler()
#trainX_scaled = scaler.fit_transform(trainX)
#scaler = RobustScaler()
#trainX_scaled = scaler.fit_transform(trainX)
#scaler = PowerTransformer()
#trainX_scaled = scaler.fit_transform(trainX)
#scaler = QuantileTransformer()
#trainX_scaled = scaler.fit_transform(trainX)

#### Train Model

In [21]:
model.fit(trainX_scaled, trainY, epochs=50, batch_size=500, verbose=0)

In [22]:
scores = model.evaluate(trainX_scaled, trainY)
print("\n%s: %.2f" % (model.metrics_names[0], scores[0]))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

25844/25844 [==============================] - 2s 73us/step

loss: 0.44

acc: 78.33%


#### Load test dataset

In [23]:
testmatchups = pandas.read_csv('kerasfiles/test.matchups.ODDeltaStats.csv')
print(testmatchups.shape)

(11390, 69)


In [24]:
testdata = testmatchups.values
testdata.shape

(11390, 69)

In [25]:
testdata = testmatchups.values

testID = testdata[:,0]                # ID for submission
testT = testdata[:,1].astype(int)     # played matchups during tourneys
testY = testdata[:,2].astype(int)     # truth (0 or 1) if played, 0 otherwise 
testX = testdata[:,3:].astype(float)  # features

print(testID.shape)
print(testT.shape)
print(testX.shape)
print(testY.shape)

(11390,)
(11390,)
(11390, 66)
(11390,)


In [26]:
# scale
testX_scaled = scaler.transform(testX)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

#### Predict test

In [ ]:
# predict all matchups
Yhat = model.predict_classes(testX_scaled)
Yprob = model.predict_proba(testX_scaled)

In [ ]:
# evaluate only played matchups 
scores = model.evaluate(testX_scaled[testT==1,:], testY[testT==1])
print("\n%s: %.2f" % (model.metrics_names[0], scores[0]))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# confusion matrix for played matchups
matrix = confusion_matrix(testY[testT==1], Yhat[testT==1])
matrix

#### Create submission file